In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 122 (delta 15), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (122/122), 1.27 MiB | 28.96 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [ ]:
cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [ ]:
!bash install_mecab-ko_on_colab_light_220429.sh

Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 39.4 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-06-02 20:59:03--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::3403:4be7, 2406:da00:ff00::22c0:3470, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?response-content-disposition=attachment%3B%20filename%3D%22mecab-0.996-ko-0.9.2.tar.gz%22&response-content-encoding=None&AWSAccessKeyId=

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from konlpy.tag import Mecab

In [ ]:
from transformers import logging
logging.set_verbosity_error()

In [ ]:
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device:", device)

model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=4,problem_type="multi_label_classification").to(device)
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

device: cuda:0


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [ ]:
model.load_state_dict(torch.load('/content/intent_model.pt'))

<All keys matched successfully>

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
mecab.morphs("1학년 1학기 IT 언제임")

['1', '학년', '1', '학기', 'IT', '언제', '임']

In [ ]:
jaccard_sim("1학년 1학기 커리큘럼 알려주셈","1학년 1학기",mecab,True)

1.0

In [ ]:
import re

stopwords_tag = [
    'IC',
    'EP', 'EF', 'EC', 'ETN', 'ETM', #어미
    'MAG', 'MAJ',#일반부사
    'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ','JX', 'JC', #조사들
    'SF','SE','SSO','SSC','SC', 'SY',
    'SL','SH' ,'SN'#외국어, 한자 ,숫자
]

class SentecePP(Dataset):

  def __init__(self, sentence_list, usr_dict=None, stopwords_tag=None, stopwords_str=None):
    self.sentence_list = sentence_list
    self.mecab = Mecab() #mecab
    self.usrdict = usr_dict
    self.stopwords_tag = stopwords_tag
    self.stopwords_str = stopwords_tag

  def __len__(self):
    return len(self.sentence_list)

  def __getitem__(self, idx):
    sentence = self.sentence_list[idx][0]
    label = self.sentence_list[idx][1]

    pattern = re.compile(r'[ㄱ-ㅎㅏ-ㅣ]+')
    sentence = re.sub(pattern, '', sentence)
    pp_sentence = ""

    for n,pos in self.mecab.pos(sentence):
        if all(pos in self.stopwords_tag for pos in pos.split('+')):
            continue
        if pos in self.stopwords_tag:
            continue
        if self.stopwords_str != None and n in self.stopwords_str:
            continue
        pp_sentence += n

    tk_dict = tokenizer(
        pp_sentence,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=128,
        padding=True
    )

    input_ids, attention_mask = tk_dict['input_ids'][0], tk_dict['attention_mask'][0]
    return input_ids, attention_mask,label


In [ ]:
df = pd.read_csv('/content/intent_sentences.csv')
len(df)

133792

In [ ]:
import re

mecab = Mecab()

stopwords_tag = [
    'IC',
    'EP', 'EF', 'EC', 'ETN', 'ETM', #어미
    'MAG', 'MAJ',#일반부사
    'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ','JX', 'JC', #조사들
    'SF','SE','SSO','SSC','SC', 'SY',
    'SL','SH' ,'SN'#외국어, 한자 ,숫자
]


In [ ]:

s = []
lab0 = []
lab1 = []
lab2 = []
lab3 = []

for i in df.values:
  sentence = i[0]
  pattern = re.compile(r'[ㄱ-ㅎㅏ-ㅣ]+')
  sentence = re.sub(pattern, '', sentence)

  pp_sentence = ""
  for n,pos in mecab.pos(sentence):
      if all(pos in stopwords_tag for pos in pos.split('+')):
          continue
      if pos in stopwords_tag:
          continue
      pp_sentence += n


  if len(pp_sentence)!=0:
    s.append([pp_sentence,i[1]])

for i in s:
  if i[1]==0:
    lab0.append(i[0])
  elif i[1]==1:
    lab1.append(i[0])
  elif i[1]==2:
    lab2.append(i[0])
  elif i[1]==3:
    lab3.append(i[0])
  continue

lab0 = list(set(tuple(lab0)))
lab1 = list(set(tuple(lab1)))
lab2 = list(set(tuple(lab2)))
lab3 = list(set(tuple(lab3)))

lab0 = [[s,0] for s in lab0[:10000]]
lab1 = [[s,1] for s in lab1[:10000]]
lab2 = [[s,2] for s in lab2[:10000]]
lab3 = [[s,3] for s in lab3[:10000]]
lst = lab0+lab1+lab2+lab3


In [ ]:
df_s = pd.DataFrame(lst)
df_s.columns = ['sentence','label']
train_data = df_s.sample(frac=0.8, random_state=42)
test_data = df_s.drop(train_data.index)
train_data

,sentence,label
32823,해야하하싫게자격증공부,3
16298,하늘공원거기어디야,1
28505,심심할때한번씩보시간흐르키키,2
6689,전화불편해요,0
26893,넌언제라햇,2
...,...,...
35414,그거로스쿨가변호사시험준비해야하거알,3
25738,구글실시간웹서치제공하서비스이름,2
32713,시험기간공부재끼볼정도재밌저번티비하거봤,3
21669,맡시간걸려,2


In [ ]:
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item = {key: val[idx].detach().clone() for key, val in self.encodings.items()}
        item = {'input_ids' : item['input_ids'], 'token_type_ids' : item['token_type_ids'] , 'attention_mask' : item['attention_mask'], "label" : torch.tensor(self.labels[idx])}
        # item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
train_label = train_data["label"].values
test_label = test_data["label"].values

tokenized_train_sentences = tokenizer(
    list(train_data['sentence']),
    return_tensors='pt',
    truncation=True,
    max_length=256,
    padding = "max_length",
    add_special_tokens=True
)

tokenized_test_sentences = tokenizer(
    list(test_data['sentence']),
    return_tensors='pt',
    truncation=True,
    max_length=256,
    padding = "max_length",
    add_special_tokens=True
)



In [ ]:
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [ ]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=4,problem_type="multi_label_classification").to(device)

In [ ]:
epochs = 3
batch_size = 16
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)


from tqdm.notebook import tqdm
from torch.nn import functional as F
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
losses = []
accuracies = []

min_loss = 3

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0
  batch_loss = 0

  model.train()

  for i in tqdm(train_loader):
    input_ids_batch, attention_masks_batch, y_batch = i['input_ids'],i['attention_mask'],i['label']
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids=input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device)).logits
    loss = criterion(y_pred, y_batch)
    loss.backward()
    optimizer.step()
    batch_loss += loss.item()
    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print(f"Batch Loss: {batch_loss:.3f} Accuracy: {float(correct.item()) / total *100:6.2f}%")
      if min_loss > batch_loss:
        min_loss = batch_loss
        torch.save(model.state_dict(), "/content/model/intent_model.pt")
      batch_loss = 0


  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print(f"Train Loss: {total_loss:.3f} Accuracy:{float(correct.item()) / total *100:6.2f}%")

  0%|          | 0/2000 [00:00<?, ?it/s]

Batch Loss: 2.919 Accuracy:  99.31%
Batch Loss: 2.808 Accuracy:  99.34%
Batch Loss: 2.856 Accuracy:  99.31%
Batch Loss: 2.612 Accuracy:  99.34%
Batch Loss: 2.534 Accuracy:  99.30%
Batch Loss: 4.026 Accuracy:  99.26%
Batch Loss: 3.348 Accuracy:  99.22%
Batch Loss: 1.944 Accuracy:  99.27%
Batch Loss: 2.289 Accuracy:  99.26%
Batch Loss: 2.960 Accuracy:  99.26%
Batch Loss: 2.234 Accuracy:  99.28%
Batch Loss: 3.085 Accuracy:  99.28%
Batch Loss: 3.254 Accuracy:  99.26%
Batch Loss: 2.529 Accuracy:  99.28%
Batch Loss: 2.425 Accuracy:  99.29%
Batch Loss: 1.957 Accuracy:  99.29%
Batch Loss: 2.250 Accuracy:  99.30%
Batch Loss: 3.241 Accuracy:  99.29%


KeyboardInterrupt: ignored

In [ ]:
#저장한 모델 불러오기
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=4,problem_type="multi_label_classification").to(device)
model.load_state_dict(torch.load('/content/model/intent_model.pt'))

<All keys matched successfully>

In [ ]:
model = torch.load('/content/model/intent_model.pt')

In [ ]:
def pred_sent(model,text,label=None):
  i = tokenizer(
    text,
    return_tensors='pt',
    truncation=True,
    max_length=256,
    padding = "max_length",
    add_special_tokens=True
  )
  pred = model(i['input_ids'].to(device),i['attention_mask'].to(device)).logits
  pred = pred.argmax()


  if pred == 0:
    pred = "[번호에 관한 질문]"
  elif pred ==1:
    pred = "[장소에 관한 질문]"
  elif pred ==2:
    pred ="[시간에 관한 질문]"
  else:
    pred = "[공부에 관한 질문]"


  if label == None:
    return f"예측 의도: {pred}"

  elif label != None:
    if pred == label:
      iscor = "😀"
    else :
      iscor = "😱"
    return f"문장 : {text}\n예측 의도: {pred} : {iscor}"




In [ ]:
 """
  if pred == 0:
    pred= "번호에 관한 질문"
  elif pred ==1:
    pred= "장소에 관한 질문"
  elif pred ==2:
    pred="시간에 관한 질문"
  else:
    pred= "공부에 관한 질문"
"""
문장 예시

print(pred_sent(model,"소학 과사 번호알려줘",0))
print("="*45)
print(pred_sent(model,"도서관 위치알려줘",1))
print("="*45)
print(pred_sent(model,"도서관 사용 가능 시간 알려줘",2))
print("="*45)
print(pred_sent(model,"이번학기 학점이 4.3인데 장금을 받을 수 있을까?",3))

문장 : 소학 과사 번호알려줘
예측 의도: [번호에 관한 질문] : 😀
문장 : 도서관 위치알려줘
예측 의도: [장소에 관한 질문] : 😀
문장 : 도서관 사용 가능 시간 알려줘
예측 의도: [시간에 관한 질문] : 😀
문장 : 이번학기 학점이 4.3인데 장금을 받을 수 있을까?
예측 의도: [공부에 관한 질문] : 😀


In [ ]:
while 1:
  print("의도 분류 문장 입력: ")
  print(pred_sent(model, input()))

의도 분류 문장 입력: 
다음학기 소학 커리큘럼 알려줘
예측 의도: [공부에 관한 질문]
의도 분류 문장 입력: 
2학년 소학 커리큘럼 알려줘
예측 의도: [공부에 관한 질문]
의도 분류 문장 입력: 
2학년 1학기 커리큘럼 알려줘
예측 의도: [공부에 관한 질문]
의도 분류 문장 입력: 
똥마려운데 어떻게해
예측 의도: [시간에 관한 질문]
의도 분류 문장 입력: 
똥마려운데 어디로 가야돼
예측 의도: [장소에 관한 질문]
의도 분류 문장 입력: 
과사 순서 알려줘
예측 의도: [시간에 관한 질문]
의도 분류 문장 입력: 
과사 번호 알려줘
예측 의도: [번호에 관한 질문]
의도 분류 문장 입력: 


KeyboardInterrupt: ignored

In [ ]:
print(pred_sent(model,"장학금에 대해 정보를 알고 싶은데 어디에 전화해야돼?",0))
print("="*60)
print(pred_sent(model,"공부하고 싶은데 어떻게 할까?",1))
print("="*60)
print(pred_sent(model,"도서관 사용 가능 시간 알려줘",2))
print("="*60)
print(pred_sent(model,"장학금 1/4 금액 얼마임?",3))